In [65]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# AI-Powered Personalized Mental Wellness Path Generator - [Ayush Rana]

**Project Overview:**
This Kaggle Notebook presents an AI-powered solution for generating personalized learning paths focused on mental well-being. Recognizing the overwhelming nature of available mental health resources, this project utilizes Google's Gemini 1.5 Flash model to create structured and tailored guidance based on individual needs and preferences. As an individual project for the Gen AI Capstone, this notebook demonstrates the application of several key Generative AI capabilities to address a significant human need.

**Individual Contributor:** [Ayush Rana]

**Project Goal:**
The primary goal of this project is to develop an AI system that can take user input regarding their mental wellness goals (e.g., reduce stress, improve sleep, manage anxiety) and preferred learning activities (e.g., meditation, breathing exercises, reading articles) and generate a customized learning path. This path is structured into modules with specific activities, estimated time commitments, and optional links to relevant resources.

**Gen AI Capabilities Demonstrated:**
This project showcases the effective use of:
* **Structured Output (JSON):** For creating organized and machine-readable learning paths.
* **Few-shot Prompting:** To guide the Gemini model in generating the desired format and content.
* **Basic Document Understanding:** Enabling the model to interpret user-provided needs and preferences.
* **Controlled Generation:** Ensuring the generated content is relevant to the user's input.

By providing this personalized and structured approach, this project aims to empower individuals to take proactive steps towards improving their mental well-being.

## Install Libraries

In [66]:
!pip install -U -q google-generativeai

## Setup and API Key

In [67]:
import google.generativeai as genai
import json
from kaggle_secrets import UserSecretsClient

GOOGLE_API_KEY = UserSecretsClient().get_secret("GOOGLE_API_KEY")
genai.configure(api_key=GOOGLE_API_KEY)

## Initialize Model

In [68]:
model = genai.GenerativeModel('gemini-1.5-flash')

## User Input

In [69]:
## User Input
try:
    needs = input("What are your primary mental wellness goals? (e.g., reduce stress, improve sleep, manage anxiety): ")
except RuntimeError:
    needs = "reduce stress"  # Default value

try:
    preferences = input("What types of activities do you prefer? (e.g., meditation, breathing exercises, reading articles): ")
except RuntimeError:
    preferences = "breathing exercises"  # Default value

user_profile = {
    "needs": needs,
    "preferences": preferences
}
print("\nYour mental wellness preferences:", user_profile)

What are your primary mental wellness goals? (e.g., reduce stress, improve sleep, manage anxiety):  reduce stress
What types of activities do you prefer? (e.g., meditation, breathing exercises, reading articles):  meditation



Your mental wellness preferences: {'needs': 'reduce stress', 'preferences': 'meditation'}


## Craft the Prompt

In [70]:
prompt = f"""
You are an AI mental wellness coach. Generate a personalized learning path for someone with the following needs and preferences:
Needs: {user_profile['needs']}
Preferences: {user_profile['preferences']}

The learning path should be outputted in JSON format with the following keys:
"title": "Title of the learning path",
"modules": [
  {{
    "module_name": "Name of the module",
    "activities": [
      {{
        "activity_name": "Name of the activity",
        "description": "Brief description",
        "estimated_time": "Estimated time (minutes)",
        "resource_link": "Optional link to a resource"
      }}
    ]
  }}
]

Here are a few examples of the desired output format:

{{
  "title": "Stress Reduction and Mindfulness Journey",
  "modules": [
    {{
      "module_name": "Week 1: Introduction to Mindfulness",
      "activities": [
        {{
          "activity_name": "Guided Breathing Meditation (5 min)",
          "description": "A short guided session to focus on your breath.",
          "estimated_time": "5",
          "resource_link": "https://example.com/breathing"
        }},
        {{
          "activity_name": "Reading: What is Mindfulness?",
          "description": "An introductory article on the benefits of mindfulness.",
          "estimated_time": "15",
          "resource_link": "https://example.com/mindfulness_article"
        }}
      ]
    }},
    {{
      "module_name": "Week 2: Managing Stress with Meditation",
      "activities": [
        {{
          "activity_name": "Body Scan Meditation (10 min)",
          "description": "A guided meditation to increase body awareness.",
          "estimated_time": "10",
          "resource_link": "https://example.com/body_scan"
        }}
      ]
    }}
  ]
}}

Generate a personalized learning path for the user's needs and preferences. Focus on activities they would find helpful based on their input.
"""

## Generate Personalized Learning Path and Parse JSON

In [71]:
try:
    response = model.generate_content(prompt)
    wellness_path_json_str = response.text.strip()
    start_index = wellness_path_json_str.find('{')
    end_index = wellness_path_json_str.rfind('}')
    if start_index != -1 and end_index != -1 and start_index < end_index:
        valid_json_str = wellness_path_json_str[start_index:end_index+1]
        wellness_path_data = json.loads(valid_json_str)

        print("\nGenerated Personalized Mental Wellness Path:")
        print(f"Title: {wellness_path_data.get('title', 'Personal Wellness Plan')}")
        for module in wellness_path_data.get('modules', []):
            print(f"\nModule: {module.get('module_name', 'Unnamed Module')}")
            for activity in module.get('activities', []):
                print(f"  - Activity: {activity.get('activity_name', 'Unnamed Activity')}")
                print(f"    Description: {activity.get('description', 'No description')}")
                print(f"    Estimated Time: {activity.get('estimated_time', 'N/A')} minutes")
                if activity.get('resource_link'):
                    print(f"    Resource: {activity.get('resource_link')}")

    else:
        print("Could not find valid JSON structure in the response.")
        print("Raw response:", wellness_path_json_str)

except json.JSONDecodeError:
    print("Error decoding JSON response.")
    print("Raw response:", response.text)
except Exception as e:
    print(f"An error occurred: {e}")


Generated Personalized Mental Wellness Path:
Title: Mindful Stress Reduction: A Meditation-Based Approach

Module: Week 1: Foundations of Mindfulness Meditation
  - Activity: Introduction to Mindfulness Meditation (Video)
    Description: A short video explaining the principles and benefits of mindfulness meditation for stress reduction.
    Estimated Time: 10 minutes
    Resource: https://www.youtube.com/watch?v=inJjDh9k_q8
  - Activity: Mindful Breathing Exercise (5 min)
    Description: Guided meditation focusing on breath awareness to calm the nervous system.
    Estimated Time: 5 minutes
    Resource: https://www.headspace.com/meditation/breathing
  - Activity: Mindful Body Scan Meditation (10 min)
    Description: Guided meditation to bring awareness to sensations in the body, releasing tension.
    Estimated Time: 10 minutes
    Resource: https://www.youtube.com/results?search_query=guided+body+scan+meditation
  - Activity: Journaling Reflection
    Description: Reflect on your

## Gen AI Capabilities Demonstrated:

This project effectively demonstrates the following Gen AI capabilities:

* **Structured Output (JSON):** The personalized learning path is generated in a structured JSON format, making it easy to understand and potentially integrate into other applications.
* **Few-shot Prompting:** The prompt includes an example of a mental wellness learning path in JSON format, guiding the Gemini model on the desired output structure and content.
* **Document Understanding (Basic):** The prompt implicitly asks the model to understand the user's needs and preferences (expressed in natural language) and tailor the learning path accordingly, demonstrating a basic form of document understanding.
* **Controlled Generation:** The user's input about their needs and preferred activities directly influences the generated learning path, demonstrating controlled generation.

## Blog Post Idea: AI for Mental Well-being: Personalized Learning Paths

**Introduction:**
In today's fast-paced world, prioritizing mental well-being is more crucial than ever. This project explores how the power of Artificial Intelligence, specifically Google's Gemini model, can be harnessed to create personalized learning paths for individuals seeking to improve their mental health.

**The Challenge of Finding the Right Mental Wellness Resources:**
The internet is flooded with information on mental wellness – from meditation apps to countless articles. However, finding a structured and personalized approach that caters to individual needs and preferences can be overwhelming and time-consuming.

**Our AI-Powered Solution: Personalized Learning Paths:**
To address this challenge, we developed an AI-powered system that generates customized learning paths for mental well-being. By taking into account a user's specific goals (e.g., reducing stress, improving sleep, managing anxiety) and preferred types of activities (e.g., meditation, breathing exercises, reading), our AI creates a structured journey with suggested activities and resources.

**How Gemini Makes it Possible:**
Leveraging the advanced capabilities of Google's Gemini model, our system demonstrates:
* **Understanding User Needs:** The AI interprets user-provided goals and preferences to tailor the learning path.
* **Structured Content Generation:** The learning path is generated in a clear, organized format with modules and specific activities.
* **Personalized Recommendations:** The AI suggests relevant activities and can even incorporate links to external resources.

**Potential Impact and Future Directions:**
This project showcases the exciting potential of AI in making mental wellness support more accessible and personalized. Future developments could include integration with existing mental health platforms, progress tracking, and more sophisticated methods for tailoring content based on user feedback.

**Conclusion:**
By providing personalized guidance, AI can empower individuals to take proactive steps towards improving their mental well-being, making the journey less daunting and more effective.

## YouTube Video Idea: AI Creates Your Personalized Mental Wellness Journey

**(Short Demo Video)**

**Video Title:** AI Crafts Your Mental Wellness Plan! Personalized Learning Paths with Gemini

**Video Script Outline:**

**(0:00-0:15) Introduction:**
* Engaging intro music and visuals related to mental wellness.
* Briefly introduce the project: an AI that creates personalized mental wellness learning paths.
* Mention the use of Google's Gemini model.

**(0:15-0:45) Demonstration:**
* Show the Kaggle Notebook interface.
* Demonstrate the user input process: entering needs (e.g., "reduce anxiety") and preferences (e.g., "breathing exercises, short articles").
* Run the code cell to generate the learning path.
* Show the generated output in a clear and easy-to-read format, highlighting the modules and activities.

**(0:45-1:15) Explanation of Key Features:**
* Briefly explain how the AI personalizes the learning path based on user input.
* Mention the structured format (modules, activities, estimated time, potential resources).
* Highlight the Gen AI capabilities at play (structured output, few-shot prompting – if briefly understandable visually through the prompt example, basic understanding of user text, controlled generation).

**(1:15-1:45) Potential Benefits and Future Scope:**
* Discuss the potential benefits for users seeking mental wellness support (personalized guidance, structured approach).
* Briefly touch upon future possibilities (integration with apps, progress tracking).

**(1:45-2:00) Conclusion and Call to Action:**
* Thank viewers for watching.
* Encourage them to check out the Kaggle Notebook (if you plan to share a link).
* End with a positive message about mental well-being.